In [ ]:
#Importing data from local computer

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving alejandroFlores_double_tap.csv to alejandroFlores_double_tap (1).csv
Saving alejandroFlores_fingers_spread.csv to alejandroFlores_fingers_spread (1).csv
Saving alejandroFlores_fist.csv to alejandroFlores_fist (1).csv
Saving alejandroFlores_wave_in.csv to alejandroFlores_wave_in (1).csv
Saving alejandroFlores_wave_out.csv to alejandroFlores_wave_out (1).csv
Saving alexandraApellido_double_tap.csv to alexandraApellido_double_tap (1).csv
Saving alexandraApellido_fingers_spread.csv to alexandraApellido_fingers_spread (1).csv
Saving alexandraApellido_fist.csv to alexandraApellido_fist (1).csv
Saving alexandraApellido_wave_in.csv to alexandraApellido_wave_in (1).csv
Saving alexandraApellido_wave_out.csv to alexandraApellido_wave_out (1).csv
Saving andresGuerra_double_tap.csv to andresGuerra_double_tap (1).csv
Saving andresGuerra_fingers_spread.csv to andresGuerra_fingers_spread (1).csv
Saving andresGuerra_fist.csv to andresGuerra_fist (1).csv
Saving andresGuerra_wave_in.csv to andresG

In [ ]:
!pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512616 sha256=ef850885693dc8718565acc2673f1cefe528bff8a397a908c5dc4bfb51eb6bc8
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.4 MB/s eta 0:00:00


In [ ]:
# aux_fnc

from scipy import signal
import numpy as np
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

def trial(data,flag_index,i_trial):

  data_trial = data[flag_index[i_trial]+1:flag_index[i_trial+1]]

  return data_trial

#Function that rectifies and filters the signal
def filt_emg(emgs_raw,b,a):

  emg_filt = []

  for emg_raw in emgs_raw:
    #Signal normalized
    if np.max(abs(emg_raw))>1:
      emg_norm = emg_raw/128
    else:
      emg_norm = emg_raw

    #Signal rectified (R)
    emg_rect = abs(emg_norm)

    #Signal filtered (V)
    emg_filt.append(signal.filtfilt(b,a,emg_rect,axis=0))

  return emg_filt
################################################################################

#Function that detect the muscle activity
def muscle_activity(emgs_filt,fs,window,numFreqOfSpec,nOver_win,hamm_win,tau_u,min_seg):

  emgs_ts         = []
  gesture_index_s = []
  gesture_index_e = []

  tau_u_aux   = 0
  tau_u_count = 0
  for emg_filt in emgs_filt:
    #Sum along the rows (S)
    emg_sum = np.sum(emg_filt,axis=-1)

    #Spectrogram (P_C)
    f,t,spec = signal.stft(emg_sum,fs=fs,window=window,nperseg=numFreqOfSpec,noverlap=nOver_win,nfft=numFreqOfSpec*2,boundary=None, padded=False)

    #Modulus (P)
    spec_abs = ((hamm_win+1)/2)*abs(spec)

    #Sum along the rows (U)
    spec_sum = np.sum(spec_abs,0)

    # print(spec_sum)
    # input("Press Enter")

    #Muscle activity detection
    great_than_tau = spec_sum >= tau_u
    great_than_tau = np.concatenate([[0],great_than_tau.astype(int),[0]])

    diff_great_than_tau = abs(np.diff(great_than_tau))

    if diff_great_than_tau[-1]==1:
      diff_great_than_tau[-2]=1

    diff_great_than_tau= diff_great_than_tau[:-1]

    index_nonzero = np.where(diff_great_than_tau==1)
    index_nonzero = np.array(index_nonzero[0])

    index_samp = np.floor(t*fs)-1

    nindex_nonzero = len(index_nonzero)

    #Where the muscle activity starts and ends
    #First condition: none activity is detected
    #Second condition: only the start is detected
    #Third condition: the start and end is detected
    if nindex_nonzero==0:
      index_s = 0
      index_e = len(emg_sum)
    elif nindex_nonzero==1:
      index_s = index_samp[index_nonzero].astype(int)
      index_e = len(emg_sum)
    else:
      index_s = index_samp[index_nonzero[0]].astype(int)
      index_e = index_samp[index_nonzero[-1]-1].astype(int)+1
    #end

    nxtra_samples = 25;
    index_s = np.maximum(0, index_s - nxtra_samples)
    index_e = np.minimum(len(emg_sum), index_e + nxtra_samples)

    #Check if the length of the activity is greater than "min_seg"
    #Else, all the signal is selected
    if (index_e - index_s) < min_seg:
      index_s = 0
      index_e = len(emg_sum)
    #end

    gesture_index_s = np.append(gesture_index_s,index_s)
    gesture_index_e = np.append(gesture_index_e,index_e)
    emgs_ts.append(emg_filt[index_s:index_e,:])
    # plt.plot(emg_filt[index_s:index_e,:])
    # plt.show()
    # input("Press Enter to continue")
  #end

  return emgs_ts, spec, spec_abs, gesture_index_s.astype(int), gesture_index_e.astype(int)
################################################################################

#Compute the best centers of the class
def best_center_class(emgs_ts):
  mtxDistances = np.zeros((len(emgs_ts), len(emgs_ts)))

  for j in range(len(emgs_ts)):
    for k in range(len(emgs_ts)):
      if k>j:
        dist_dtw, path_dtw = fastdtw(emgs_ts[j],emgs_ts[k],radius = 100,dist=2)
        mtxDistances[j,k] = dist_dtw
        mtxDistances[k,j] = dist_dtw
      #end

    #end

  #end

  vectDistances = np.sum(mtxDistances,0)
  idx_min       = np.argmin(vectDistances)
  center_idx    = emgs_ts[idx_min]
  # print(mtxDistances,vectDistances,idx_min,center_idx, sep='\n')

  return center_idx
################################################################################

#Compute the best centers of the class
def best_center_dtai(emgs_ts):
  mtxDistances = np.zeros((len(emgs_ts), len(emgs_ts)))

  for j in range(len(emgs_ts)):
    for k in range(len(emgs_ts)):
      if k>j:
        dist_dtw = dtw.distance_fast(emgs_ts[j],emgs_ts[k],use_ndim=True,use_c=True)
        mtxDistances[j,k] = dist_dtw
        mtxDistances[k,j] = dist_dtw
      #end

    #end

  #end

  vectDistances = np.sum(mtxDistances,0)
  idx_min       = np.argmin(vectDistances)
  center_idx    = emgs_ts[idx_min]
  # print(mtxDistances,vectDistances,idx_min,center_idx, sep='\n')

  return center_idx
################################################################################

#Feature extraction
def feat_extrac(emgs_ts,center):
  dataX_temp   = [fastdtw(emg_ts,center,radius = 100,dist=2) for emg_ts in emgs_ts]
  dataX, pathX = map(list,zip(*dataX_temp))

  return dataX
################################################################################

# DTW feature extraction
def dtw_extract(emgs_ts,center):
  dataX_temp   = [fastdtw(emg_ts,center,radius = 100,dist=2) for emg_ts in emgs_ts]
  dataX, pathX = map(list,zip(*dataX_temp))

  return dataX
################################################################################

# Dtai DTW extraction
def dtaidtw_extract(emgs_ts, center):
  dataX_temp = [dtw.distance_fast(emg_ts, center, use_ndim=True, use_c=True) for emg_ts in emgs_ts]

  return dataX_temp
################################################################################

#Feature normalization
def feat_norm(dataX):
  df_dataX      = dataX.copy()
  df_dataX_mean = pd.DataFrame(df_dataX.mean(axis=1),columns=['Mean'])
  df_dataX_std  = pd.DataFrame(df_dataX.std(axis=1),columns=['Std'])


  normalized_feat = df_dataX.sub(df_dataX_mean['Mean'],axis=0)
  normalized_feat = normalized_feat.div(df_dataX_std['Std'],axis=0)

  return normalized_feat
################################################################################

# Feature normalization
def feat_normalization(data):
  row_means = np.mean(data, axis=1, keepdims=True)
  row_stds  = np.std(data, axis=1, keepdims=True, ddof=1)

  data_normalized = (data - row_means) / row_stds

  return data_normalized
################################################################################

#Normalized Least Mean Square NLMS
def nlms(d,x,mu,N,init,gamma):
  nCoefficients = N+1
  nIterations   = np.size(d,1)

  errVector         = np.zeros((nIterations, 1),dtype='float64')
  outputVector      = np.zeros((nIterations, 1),dtype='float64')
  coefficientVector = np.zeros((nCoefficients, nIterations+1),dtype='float64')

  coefficientVector[:,0] = init.flatten()

  prefixedInput = np.vstack(((np.zeros((nCoefficients-1,1),dtype='float64')),np.reshape(x,(len(x),1))))
  # print(nCoefficients)
  # print(nIterations)
  # print(np.shape(prefixedInput))
  for it in range(nIterations):

    regressor = np.flipud(prefixedInput[it:it + (nCoefficients-1)+1])
    # regressor = prefixedInput[-(it + (nCoefficients-1)+1):it:-1]

    outputVector[it] = np.dot(coefficientVector[:,it].T,regressor)
    # print(np.shape(d))
    errVector[it] = d[:,it]-outputVector[it]
    # print(errVector[it])

    # if it <= 400:
    #   coefficientVector[:,it+1] = coefficientVector[:,it]+((mu/(gamma + np.dot(regressor.T,regressor)))*np.conj(errVector[it])*regressor).flatten()
    # else:
    #   coefficientVector[:,it+1] = coefficientVector[:,it]
    coefficientVector[:,it+1] = coefficientVector[:,it]+((mu/(gamma + np.dot(regressor.T,regressor)))*np.conj(errVector[it])*regressor).flatten()

  return outputVector, errVector, coefficientVector

In [ ]:
#neural_net
import numpy as np
from scipy.spatial.distance import euclidean

# Initialize parameters
def initialize_parameters(layers_dims):
  """
  Initialize parameters dictionary.

  Weight matrices will be initialized to random values from uniform normal
  distribution.
  bias vectors will be initialized to zeros.

  Arguments
  ---------
  layers_dims : list or array-like
      dimensions of each layer in the network.

  Returns
  -------
  parameters : dict
      weight matrix and the bias vector for each layer.
  """
  # np.random.seed(1)
  parameters = {}
  L = len(layers_dims)

  for l in range(1, L):
    r = np.sqrt(6) / np.sqrt(layers_dims[l] + layers_dims[l - 1] + 1)
    W = np.random.randn(layers_dims[l], layers_dims[l - 1])*2*r - r
    parameters["W" + str(l)] = W
    parameters["b" + str(l)] = np.ones((layers_dims[l], 1))

    assert parameters["W" + str(l)].shape == (layers_dims[l], layers_dims[l - 1]), parameters["W" + str(l)].shape
    # assert parameters["b" + str(l)].shape == (layers_dims[l], 1)

  return parameters

# Define activation functions that will be used in forward propagation
def sigmoid(Z):
  """
  Computes the sigmoid of Z element-wise.

  Arguments
  ---------
  Z : array
      output of affine transformation.

  Returns
  -------
  A : array
      post activation output.
  Z : array
      output of affine transformation.
  """
  A = 1 / (1 + np.exp(-Z))

  return A, Z


def tanh(Z):
  """
  Computes the Hyperbolic Tagent of Z elemnet-wise.

  Arguments
  ---------
  Z : array
      output of affine transformation.

  Returns
  -------
  A : array
      post activation output.
  Z : array
      output of affine transformation.
  """
  A = np.tanh(Z)

  return A, Z


def relu(Z):
  """
  Computes the Rectified Linear Unit (ReLU) element-wise.

  Arguments
  ---------
  Z : array
      output of affine transformation.

  Returns
  -------
  A : array
      post activation output.
  Z : array
      output of affine transformation.
  """
  A = np.maximum(0, Z)

  return A, Z


def leaky_relu(Z):
  """
  Computes Leaky Rectified Linear Unit element-wise.

  Arguments
  ---------
  Z : array
      output of affine transformation.

  Returns
  -------
  A : array
      post activation output.
  Z : array
      output of affine transformation.
  """
  A = np.maximum(0.1 * Z, Z)

  return A, Z

# Define helper functions that will be used in L-model forward prop
def linear_forward(A_prev, W, b):
  """
  Computes affine transformation of the input.

  Arguments
  ---------
  A_prev : 2d-array
      activations output from previous layer.
  W : 2d-array
      weight matrix, shape: size of current layer x size of previuos layer.
  b : 2d-array
      bias vector, shape: size of current layer x 1.

  Returns
  -------
  Z : 2d-array
      affine transformation output.
  cache : tuple
      stores A_prev, W, b to be used in backpropagation.
  """
  Z = np.dot(W, A_prev) + b
  cache = (A_prev, W, b)

  return Z, cache


def linear_activation_forward(A_prev, W, b, activation_fn):
  """
  Computes post-activation output using non-linear activation function.

  Arguments
  ---------
  A_prev : 2d-array
      activations output from previous layer.
  W : 2d-array
      weight matrix, shape: size of current layer x size of previuos layer.
  b : 2d-array
      bias vector, shape: size of current layer x 1.
  activation_fn : str
      non-linear activation function to be used: "sigmoid", "tanh", "relu".

  Returns
  -------
  A : 2d-array
      output of the activation function.
  cache : tuple
      stores linear_cache and activation_cache. ((A_prev, W, b), Z) to be used in backpropagation.
  """
  # assert activation_fn == "sigmoid" or activation_fn == "tanh" or \
  #     activation_fn == "relu"

  if activation_fn == "sigmoid":
      Z, linear_cache = linear_forward(A_prev, W, b)
      A, activation_cache = sigmoid(Z)

  elif activation_fn == "tanh":
      Z, linear_cache = linear_forward(A_prev, W, b)
      A, activation_cache = tanh(Z)

  elif activation_fn == "relu":
      Z, linear_cache = linear_forward(A_prev, W, b)
      A, activation_cache = relu(Z)

  elif activation_fn == "softmax":
      Z, linear_cache = linear_forward(A_prev, W, b)
      Zaux = Z - np.amax(Z,0)
      Zaux = np.exp(Zaux)
      A = Zaux/np.sum(Zaux,0)
      activation_cache = Zaux

  assert A.shape == (W.shape[0], A_prev.shape[1])

  cache = (linear_cache, activation_cache)

  return A, cache


def L_model_forward( X, parameters, hidden_layers_activation_fn="relu"):
  """
  Computes the output layer through looping over all units in topological
  order.

  Arguments
  ---------
  X : 2d-array
      input matrix of shape input_size x training_examples.
  parameters : dict
      contains all the weight matrices and bias vectors for all layers.
  hidden_layers_activation_fn : str
      activation function to be used on hidden layers: "tanh", "relu".

  Returns
  -------
  AL : 2d-array
      probability vector of shape 1 x training_examples.
  caches : list
      that contains L tuples where each layer has: A_prev, W, b, Z.
  """
  A = X
  caches = []
  L = len(parameters) //2

  for l in range(1, L):
    A_prev = A
    A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], activation_fn=hidden_layers_activation_fn)
    caches.append(cache)


  AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation_fn="softmax")
  caches.append(cache)

  # assert AL.shape == (1, X.shape[1])

  return AL, caches

# Compute cross-entropy cost
def compute_cost(AL, y, W, numLayers, lambda_reg):
  """
  Computes the binary Cross-Entropy cost.

  Arguments
  ---------
  AL : 2d-array
      probability vector of shape 1 x training_examples.
  y : 2d-array
      true "label" vector.

  Returns
  -------
  cost : float
      binary cross-entropy cost.
  """
  m = y.shape[1]
  # cost = - (1 / m)*np.sum(np.multiply(y, np.log(AL + 1e-30)) + np.multiply(1 - y, np.log(1 - AL)))
  cost = - (1 / m)*np.sum(np.multiply(y, np.log(AL + np.exp(-30))))
  regularization = 0
  for i in range(1,numLayers+1):
    regularization += np.sum(np.sum(np.square(W['W'+str(i)])))

  cost += (1 / m)*(lambda_reg/2)*regularization

  return cost

# Define derivative of activation functions w.r.t z that will be used in back-propagation
def sigmoid_gradient(dA, Z):
  """
  Computes the gradient of sigmoid output w.r.t input Z.

  Arguments
  ---------
  dA : 2d-array
      post-activation gradient, of any shape.
  Z : 2d-array
      input used for the activation fn on this layer.

  Returns
  -------
  dZ : 2d-array
      gradient of the cost with respect to Z.
  """
  A, Z = sigmoid(Z)
  dZ = dA * A * (1 - A)

  return dZ


def tanh_gradient(dA, Z):
  """
  Computes the gradient of hyperbolic tangent output w.r.t input Z.

  Arguments
  ---------
  dA : 2d-array
      post-activation gradient, of any shape.
  Z : 2d-array
      input used for the activation fn on this layer.

  Returns
  -------
  dZ : 2d-array
      gradient of the cost with respect to Z.
  """
  A, Z = tanh(Z)
  dZ = dA * (1 - np.square(A))

  return dZ


def relu_gradient(dA, Z):
  """
  Computes the gradient of ReLU output w.r.t input Z.

  Arguments
  ---------
  dA : 2d-array
      post-activation gradient, of any shape.
  Z : 2d-array
      input used for the activation fn on this layer.

  Returns
  -------
  dZ : 2d-array
      gradient of the cost with respect to Z.
  """
  A, Z = relu(Z)
  dZ = np.multiply(dA, np.int64(A > 0))

  return dZ


# define helper functions that will be used in L-model back-prop
def linear_backword(dZ, cache, reg):
  """
  Computes the gradient of the output w.r.t weight, bias, and post-activation
  output of (l - 1) layers at layer l.

  Arguments
  ---------
  dZ : 2d-array
      gradient of the cost w.r.t. the linear output (of current layer l).
  cache : tuple
      values of (A_prev, W, b) coming from the forward propagation in the current layer.

  Returns
  -------
  dA_prev : 2d-array
      gradient of the cost w.r.t. the activation (of the previous layer l-1).
  dW : 2d-array
      gradient of the cost w.r.t. W (current layer l).
  db : 2d-array
      gradient of the cost w.r.t. b (current layer l).
  """
  A_prev, W, b = cache
  m = A_prev.shape[1]

  dW = (1 / m) * np.dot(dZ, A_prev.T) + (1/m)*reg*W
  db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
  dA_prev = np.dot(W.T, dZ)

  assert dA_prev.shape == A_prev.shape
  assert dW.shape == W.shape
  assert db.shape == b.shape

  return dA_prev, dW, db


def linear_activation_backward(dA, cache, activation_fn, reg):
  """
  Arguments
  ---------
  dA : 2d-array
      post-activation gradient for current layer l.
  cache : tuple
      values of (linear_cache, activation_cache).
  activation : str
      activation used in this layer: "sigmoid", "tanh", or "relu".

  Returns
  -------
  dA_prev : 2d-array
      gradient of the cost w.r.t. the activation (of the previous layer l-1), same shape as A_prev.
  dW : 2d-array
      gradient of the cost w.r.t. W (current layer l), same shape as W.
  db : 2d-array
      gradient of the cost w.r.t. b (current layer l), same shape as b.
  """
  linear_cache, activation_cache = cache

  if activation_fn == "sigmoid":
      dZ = sigmoid_gradient(dA, activation_cache)
      dA_prev, dW, db = linear_backword(dZ, linear_cache, reg)

  elif activation_fn == "tanh":
      dZ = tanh_gradient(dA, activation_cache)
      dA_prev, dW, db = linear_backword(dZ, linear_cache, reg)

  elif activation_fn == "relu":
      dZ = relu_gradient(dA, activation_cache)
      dA_prev, dW, db = linear_backword(dZ, linear_cache, reg)

  return dA_prev, dW, db


def L_model_backward(AL, y, caches, hidden_layers_activation_fn="relu", reg=1e-2):
  """
  Computes the gradient of output layer w.r.t weights, biases, etc. starting
  on the output layer in reverse topological order.

  Arguments
  ---------
  AL : 2d-array
      probability vector, output of the forward propagation (L_model_forward()).
  y : 2d-array
      true "label" vector (containing 0 if non-cat, 1 if cat).
  caches : list
      list of caches for all layers.
  hidden_layers_activation_fn :
      activation function used on hidden layers: "tanh", "relu".

  Returns
  -------
  grads : dict
      with the gradients.
  """
  y = y.reshape(AL.shape)

  L = len(caches)
  grads = {}

  # dAL = np.divide(AL - y, np.multiply(AL, 1 - AL))

  # grads["dA" + str(L - 1)], grads["dW" + str(L)], grads[
  #     "db" + str(L)] = linear_activation_backward(
  #         dAL, caches[L - 1], "sigmoid", reg)

  # for l in range(L - 1, 0, -1):
  #     current_cache = caches[l - 1]
  #     grads["dA" + str(l - 1)], grads["dW" + str(l)], grads[
  #         "db" + str(l)] = linear_activation_backward(
  #             grads["dA" + str(l)], current_cache,
  #             hidden_layers_activation_fn, reg)

  m = y.shape[1]
  Wgradient = {}
  delta = -(1/m)*(y-AL)

  for l in range(L - 1, -1, -1):
      current_cache = caches[l]
      linear_cache, activation_cache = current_cache
      A_prev, W, b = linear_cache

      Wgradient[l] = np.hstack((b,W))

      if l==(L-1):
        Wgradient[l] = np.dot(delta,np.vstack((np.ones((1,A_prev.shape[1])),A_prev)).T)
      else:
        Wgradient[l] = np.dot(delta[1:,:],np.vstack((np.ones((1,A_prev.shape[1])),A_prev)).T)

      Wgradient[l][:,1:] += (1/m)*reg*W

      grads["dW"+str(l+1)] = Wgradient[l][:,1:]
      grads["db"+str(l+1)] = np.reshape(Wgradient[l][:,0],(Wgradient[l][:,0].shape[0],1))

      if l == 0:
        break

      current_cache = caches[l-1]
      linear_cache, activation_cache = current_cache
      Z = activation_cache

      if hidden_layers_activation_fn == "tanh":
        trans_func_der = 1 - np.square(2/(1+np.exp(-2*Z))-1)

      if l < L-1:
        delta = np.multiply(delta[1:,:].T@np.hstack((b,W)),np.vstack((np.ones((1,A_prev.shape[1])),trans_func_der)).T)
      else:
        delta = np.multiply(delta.T@np.hstack((b,W)),np.vstack((np.ones((1,A_prev.shape[1])),trans_func_der)).T)

      delta = delta.T


  return grads

# define the function to update both weight matrices and bias vectors
def update_parameters(parameters, grads, learning_rate):
  """
  Update the parameters' values using gradient descent rule.

  Arguments
  ---------
  parameters : dict
      contains all the weight matrices and bias vectors for all layers.
  grads : dict
      stores all gradients (output of L_model_backward).

  Returns
  -------
  parameters : dict
      updated parameters.
  """
  L = len(parameters) // 2

  for l in range(1, L + 1):
      parameters["W" + str(l)] = parameters[
          "W" + str(l)] - learning_rate * grads["dW" + str(l)]
      parameters["b" + str(l)] = parameters[
          "b" + str(l)] - learning_rate * grads["db" + str(l)]

  return parameters

# Define the multi-layer model using all the helper functions we wrote before


def L_layer_model(X, y, layers_dims, learning_rate=0.01, num_iterations=3000, print_cost=False, hidden_layers_activation_fn="relu", lambda_reg=1e-2):
  """
  Implements multilayer neural network using gradient descent as the
  learning algorithm.

  Arguments
  ---------
  X : 2d-array
      data, shape: number of examples x num_px * num_px * 3.
  y : 2d-array
      true "label" vector, shape: 1 x number of examples.
  layers_dims : list
      input size and size of each layer, length: number of layers + 1.
  learning_rate : float
      learning rate of the gradient descent update rule.
  num_iterations : int
      number of iterations of the optimization loop.
  print_cost : bool
      if True, it prints the cost every 100 steps.
  hidden_layers_activation_fn : str
      activation function to be used on hidden layers: "tanh", "relu".

  Returns
  -------
  parameters : dict
      parameters learnt by the model. They can then be used to predict test examples.
  """
  # np.random.seed(1)

  # initialize parameters
  parameters = initialize_parameters(layers_dims)

  # intialize cost list
  cost_list = []

  ############MATLAB fmincg##############
  RHO   = 0.01                                                                     # a bunch of constants for line searches
  SIG   = 0.5                                                                      # RHO and SIG are the constants in the Wolfe-Powell conditions
  INT   = 0.1                                                                      # don't reevaluate within 0.1 of the limit of the current bracket
  EXT   = 3.0                                                                      # extrapolate maximum 3 times the current bracket
  MAX   = 20                                                                       # max 20 function evaluations per line search
  RATIO = 100                                                                      # maximum allowed slope ratio

  L = len(parameters) // 2

  red = 1
  S=['Iteration: ']
  df1_aux = np.empty(([]))
  for l in range(1, L + 1):
      df1_temp = np.hstack((parameters["b" + str(l)],parameters["W" + str(l)]))
      df1_aux = np.vstack((df1_aux,np.reshape(np.transpose(df1_temp),(np.shape(df1_temp)[0]*np.shape(df1_temp)[1],1))))
  W = df1_aux[1:]

  ####################Ler um W especifico de um .csv###############
  # path = "X.csv"

  # df = pd.read_csv(path, sep=r'\s*,\s*',header=None, encoding='ascii', engine='python')
  # df.columns = ["X1"]
  # W = df.values
  # SP = 0
  # for l in range(1, L + 1):
  #   EP = SP+np.shape(parameters["b" + str(l)])[0]
  #   parameters["b" + str(l)] = W[SP:EP]
  #   SP = EP
  #   EP = SP+np.shape(parameters["W" + str(l)])[0]*np.shape(parameters["W" + str(l)])[1]
  #   parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[0],np.shape(parameters["W" + str(l)])[1])))
  #   SP = EP
  #############################################################

  # i = 0                                                                            # zero the run length counter
  ls_failed = 0                                                                    # no previous line search has failed
  fW = np.empty(([]))

  # iterate over L-layers to get the final output and the cache
  AL, caches = L_model_forward(X, parameters, hidden_layers_activation_fn)

  # compute cost to plot it
  cost = compute_cost(AL, y, parameters, len(parameters)//2, lambda_reg)

  # iterate over L-layers backward to get gradients
  grads = L_model_backward(AL, y, caches, hidden_layers_activation_fn, lambda_reg)

  f1 = cost
  df1_aux = np.empty(([]))
  for l in range(1, L + 1):
      df1_temp = np.hstack((grads["db" + str(l)],grads["dW" + str(l)]))
      df1_aux = np.vstack((df1_aux,np.reshape(np.transpose(df1_temp),(np.shape(df1_temp)[0]*np.shape(df1_temp)[1],1))))
  df1 = df1_aux[1:]
  # print(f1)
  # print(df1)
  # input("Press Enter to continue...")
  # i = i + (length<0);                                                            # count epochs?!
  s = -df1                                                                         # search direction is steepest
  d1 = np.dot(-np.transpose(s),s)                                                                # this is the slope
  z1 = red/(1-d1)                                                                  # initial step is red/(|s|+1)

  # iterate over num_iterations
  for i in range(num_iterations):
      # # iterate over L-layers to get the final output and the cache
      # AL, caches = L_model_forward(
      #     X, parameters, hidden_layers_activation_fn)

      # # compute cost to plot it
      # cost = compute_cost(AL, y, parameters, len(parameters)//2, lambda_reg)
      # # iterate over L-layers backward to get gradients
      # grads = L_model_backward(AL, y, caches, hidden_layers_activation_fn, lambda_reg)

      # # update parameters
      # parameters = update_parameters(parameters, grads, learning_rate)

      ############MATLAB fmincg#############
      W0 = W; f0 = f1; df0 = df1                                                 # make a copy of current values
      W = W + z1*s;                                                              # begin line search

      SP = 0
      for l in range(1, L + 1):
        EP = SP+np.shape(parameters["b" + str(l)])[0]
        parameters["b" + str(l)] = W[SP:EP]
        SP = EP
        EP = SP+np.shape(parameters["W" + str(l)])[0]*np.shape(parameters["W" + str(l)])[1]
        # parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[0],np.shape(parameters["W" + str(l)])[1])))
        parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[1],np.shape(parameters["W" + str(l)])[0])))
        SP = EP

      # iterate over L-layers to get the final output and the cache
      #aqui
      AL, caches = L_model_forward(
          X, parameters, hidden_layers_activation_fn)

      # compute cost to plot it
      cost = compute_cost(AL, y, parameters, len(parameters)//2, lambda_reg)

      # iterate over L-layers backward to get gradients
      grads = L_model_backward(AL, y, caches, hidden_layers_activation_fn, lambda_reg)

      f2 = cost
      df1_aux = np.empty(([]))
      for l in range(1, L + 1):
          df1_temp = np.hstack((grads["db" + str(l)],grads["dW" + str(l)]))
          df1_aux = np.vstack((df1_aux,np.reshape(np.transpose(df1_temp),(np.shape(df1_temp)[0]*np.shape(df1_temp)[1],1))))
      df2 = df1_aux[1:]

      # i = i + (num_iterations<0);                                               # count epochs?!
      d2 = np.dot(np.transpose(df2),s);
      f3 = f1; d3 = d1; z3 = -z1;                                               # initialize point 3 equal to point 1
      if num_iterations>0:
        M = MAX
      else:
        M = min(MAX, -num_iterations-i)
      success = 0; limit = -1                                                   # initialize quanteties

      while True:
        while ((f2 > f1+z1*RHO*d1) or (d2 > -SIG*d1)) and (M > 0):
          limit = z1                                                            # tighten the bracket
          if f2 > f1:
            z2 = z3 - (0.5*d3*z3*z3)/(d3*z3+f2-f3)                              # quadratic fit
          else:
            A = 6*(f2-f3)/z3+3*(d2+d3)                                          # cubic fit
            B = 3*(f3-f2)-z3*(d3+2*d2)
            z2 = (np.sqrt(B*B-A*d2*z3*z3)-B)/A                                  # numerical error possible - ok!

          if np.isnan(z2) or np.isinf(z2):
            z2 = z3/2                                                           # if we had a numerical problem then bisect

          z2 = max(min(z2, INT*z3),(1-INT)*z3)                                  # don't accept too close to limits
          z1 = z1 + z2                                                          # update the step
          W = W + z2*s

          SP = 0
          for l in range(1, L + 1):
            EP = SP+np.shape(parameters["b" + str(l)])[0]
            parameters["b" + str(l)] = W[SP:EP]
            SP = EP
            EP = SP+np.shape(parameters["W" + str(l)])[0]*np.shape(parameters["W" + str(l)])[1]
            # parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[0],np.shape(parameters["W" + str(l)])[1])))
            parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[1],np.shape(parameters["W" + str(l)])[0])))
            SP = EP

          # iterate over L-layers to get the final output and the cache
          AL, caches = L_model_forward(X, parameters, hidden_layers_activation_fn)

          # compute cost to plot it
          cost = compute_cost(AL, y, parameters, len(parameters)//2, lambda_reg)

          # iterate over L-layers backward to get gradients
          grads = L_model_backward(AL, y, caches, hidden_layers_activation_fn, lambda_reg)

          f2 = cost
          df1_aux = np.empty(([]))
          for l in range(1, L + 1):
              df1_temp = np.hstack((grads["db" + str(l)],grads["dW" + str(l)]))
              df1_aux = np.vstack((df1_aux,np.reshape(np.transpose(df1_temp),(np.shape(df1_temp)[0]*np.shape(df1_temp)[1],1))))
          df2 = df1_aux[1:]

          M = M - 1
          # i = i + (length<0);                           % count epochs?!
          d2 = np.dot(np.transpose(df2),s)
          z3 = z3-z2                                                            # z3 is now relative to the location of z2

        if f2 > f1+z1*RHO*d1 or d2 > -SIG*d1:
          break                                                                 # this is a failure
        elif d2 > SIG*d1:
          success = 1; break                                                    # success
        elif M == 0:
          break                                                                 # failure

        A = 6*(f2-f3)/z3+3*(d2+d3)                                              # make cubic extrapolation
        B = 3*(f3-f2)-z3*(d3+2*d2)
        z2 = -d2*z3*z3/(B+np.sqrt(B*B-A*d2*z3*z3))                              # num. error possible - ok!
        if ~np.isreal(z2) or np.isnan(z2) or np.isinf(z2) or z2 < 0:            # num prob or wrong sign?
          if limit < -0.5:                                                      # if we have no upper limit
            z2 = z1 * (EXT-1)                                                   # the extrapolate the maximum amount
          else:
            z2 = (limit-z1)/2                                                   # otherwise bisect

        elif (limit > -0.5) and (z2+z1 > limit):                                # extraplation beyond max?
          z2 = (limit-z1)/2                                                     # bisect
        elif (limit < -0.5) and (z2+z1 > z1*EXT):                               # extrapolation beyond limit
          z2 = z1*(EXT-1.0)                                                     # set to extrapolation limit
        elif z2 < -z3*INT:
          z2 = -z3*INT
        elif (limit > -0.5) and (z2 < (limit-z1)*(1.0-INT)):                    # too close to limit?
          z2 = (limit-z1)*(1.0-INT)

        f3 = f2; d3 = d2; z3 = -z2                                              # set point 3 equal to point 2
        z1 = z1 + z2
        W = W + z2*s                                                            # update current estimates

        SP = 0
        for l in range(1, L + 1):
          EP = SP+np.shape(parameters["b" + str(l)])[0]
          parameters["b" + str(l)] = W[SP:EP]
          SP = EP
          EP = SP+np.shape(parameters["W" + str(l)])[0]*np.shape(parameters["W" + str(l)])[1]
          # parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[0],np.shape(parameters["W" + str(l)])[1])))
          parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[1],np.shape(parameters["W" + str(l)])[0])))
          SP = EP

        # iterate over L-layers to get the final output and the cache
        AL, caches = L_model_forward(
            X, parameters, hidden_layers_activation_fn)

        # compute cost to plot it
        cost = compute_cost(AL, y, parameters, len(parameters)//2, lambda_reg)

        # iterate over L-layers backward to get gradients
        grads = L_model_backward(AL, y, caches, hidden_layers_activation_fn, lambda_reg)

        f2 = cost
        df1_aux = np.empty(([]))
        for l in range(1, L + 1):
            df1_temp = np.hstack((grads["db" + str(l)],grads["dW" + str(l)]))
            df1_aux = np.vstack((df1_aux,np.reshape(np.transpose(df1_temp),(np.shape(df1_temp)[0]*np.shape(df1_temp)[1],1))))
        df2 = df1_aux[1:]
        M = M - 1
        # i = i + (length<0);                             % count epochs?!
        d2 = np.dot(np.transpose(df2),s)
        # end of line search

      if success:                                                               # if line search succeeded
        f1 = f2; fW = np.vstack((fW,f1))
        # print('%s %4i | Cost: %4.6e\r', S, i, f1)
        s = (np.dot(np.transpose(df2),df2)-np.dot(np.transpose(df1),df2))/(np.dot(np.transpose(df1),df1))*s - df2         # Polack-Ribiere direction
        tmp = df1; df1 = df2; df2 = tmp                                         # swap derivatives
        d2 = np.dot(np.transpose(df1),s)
        if d2 > 0:                                                              # new slope must be negative
          s = -df1                                                              # otherwise use steepest direction
          d2 = np.dot(-np.transpose(s),s)

        z1 = z1 * min(RATIO, d1/(d2-np.finfo(float).tiny))                      # slope ratio but max RATIO
        d1 = d2
        ls_failed = 0                                                           # this line search did not fail
      else:
        W = W0; f1 = f0; df1 = df0                                              # restore point from before failed line search
        if ls_failed or i > abs(num_iterations):                                # line search failed twice in a row
          break                                                                 # or we ran out of time, so we give up

        tmp = df1; df1 = df2; df2 = tmp                                         # swap derivatives
        s = -df1                                                                # try steepest
        d1 = np.dot(-np.transpose(s),s)
        z1 = 1/(1-d1)
        ls_failed = 1                                                           # this line search failed

      # append each 100th cost to the cost list
      if (i + 1) % 100 == 0 and print_cost:
          print(f"The cost after {i + 1} iterations is: {cost:.4f}")

      if i % 100 == 0:
          cost_list.append(cost)

  # plot the cost curve
  # plt.figure(figsize=(10, 6))
  # plt.plot(cost_list)
  # plt.xlabel("Iterations (per hundreds)")
  # plt.ylabel("Loss")
  # plt.title(f"Loss curve for the learning rate = {learning_rate}")

  SP = 0
  for l in range(1, L + 1):
    EP = SP+np.shape(parameters["b" + str(l)])[0]
    parameters["b" + str(l)] = W[SP:EP]
    SP = EP
    EP = SP+np.shape(parameters["W" + str(l)])[0]*np.shape(parameters["W" + str(l)])[1]
    # parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[0],np.shape(parameters["W" + str(l)])[1])))
    parameters["W" + str(l)] = np.transpose(np.reshape(W[SP:EP],(np.shape(parameters["W" + str(l)])[1],np.shape(parameters["W" + str(l)])[0])))
    SP = EP

  return parameters

# def accuracy(X, parameters, y, activation_fn="relu"):
#   """
#   Computes the average accuracy rate.

#   Arguments
#   ---------
#   X : 2d-array
#       data, shape: number of examples x num_px * num_px * 3.
#   parameters : dict
#       learnt parameters.
#   y : 2d-array
#       true "label" vector, shape: 1 x number of examples.
#   activation_fn : str
#       activation function to be used on hidden layers: "tanh", "relu".

#   Returns
#   -------
#   accuracy : float
#       accuracy rate after applying parameters on the input data
#   """
#   probs, caches = L_model_forward(X, parameters, activation_fn)
#   labels = (probs >= 0.5) * 1
#   accuracy = np.mean(labels == y) * 100

#   return f"The accuracy rate is: {accuracy:.2f}%."


In [ ]:
# data reader

import numpy as np
import pandas as pd

def data_reader(users, gestures, version):

  subjects = []

  # gestures_code = pd.DataFrame(gestures,columns=['Gestures'])

  if version == 'training':
    flag_name = ['TrialOne','TrialTwo','TrialThree','TrialFour','TrialFive']

  elif version == 'testing':
    flag_name = ['TrialOne','TrialTwo','TrialThree','TrialFour','TrialFive','TrialSix','TrialSeven','TrialEight',
                'TrialNine','TrialTen','TrialEleven','TrialTwelve','TrialThirteen','TrialFourteen','TrialFifteen',
                'TrialSixteen','TrialSeventeen','TrialEighteen','TrialNineteen','TrialTwenty','TrialTwentyOne',
                'TrialTwentyTwo','TrialTwentyThree','TrialTwentyFour','TrialTwentyFive','TrialTwentySix','TrialTwentySeven',
                'TrialTwentyEight','TrialTwentyNine','TrialThirty']

    if 'relax' in gestures:
      gestures = gestures[1:]

  else:
    raise ValueError("Parameter 'version' must be either 'training' or 'testing'.")

  for user in users:
    subject_data = []

    for gesture in gestures:
      if version == 'training':
        path = user + '_' + gesture + '.csv'
      else:
        path = user + '_' + gesture + ' (1).csv'

      df = pd.read_csv(path,
                       sep=r'\s*,\s*',
                       header=0,
                       encoding='ascii',
                       engine='python')

      flag = df.loc[df['ChannelOne'].isin(flag_name)]

      flag_index = np.array(flag.index.values)
      flag_index = np.append(flag_index,len(df))

      trial_semg = [trial(df,flag_index,i) for i in range(len(flag_index)-1)]

      subject_data.append(
          dict(
              gesture = gesture,
              trials = trial_semg
          )
      )

    subjects.append(
        dict(
            name = user,
            data = subject_data
        )
    )

  return subjects

In [ ]:
# set split

import numpy as np
import copy
import random

def set_split(data_train, data_test, validation, val_split):

  N_users = len(data_train) # Acquire the quantity of users in the dataset

  data_splitted = []

  if validation == 'mhold':

    for subject_i in range(N_users):

      train_data_splitted = []
      val_data_splitted   = []
      test_data_splitted  = []

      for gesture_data in data_train[subject_i]['data']:
        train_data = [trial_data.to_numpy().astype(float) for trial_data in gesture_data['trials']]

        train_data_splitted.append(
            dict(
                gesture = gesture_data['gesture'],
                data = train_data
            )
        )

      for gesture_data in data_test[subject_i]['data']:
        val_len = val_split*len(gesture_data['trials'])
        val_len = int(np.floor(val_len))

        val_data  = [trial_data.to_numpy().astype(float) for trial_data in gesture_data['trials'][:val_len]]

        val_data_splitted.append(
            dict(
                gesture = gesture_data['gesture'],
                data = val_data
            )
        )
        test_data = [trial_data.to_numpy().astype(float) for trial_data in gesture_data['trials'][val_len:]]

        test_data_splitted.append(
            dict(
                gesture = gesture_data['gesture'],
                data = test_data
            )
        )

      data_splitted.append(
          dict(
              name = data_train[subject_i]['name'],
              train = train_data_splitted,
              val = val_data_splitted,
              test = test_data_splitted
          )
      )

    return data_splitted

In [ ]:
# preproc new

# from scipy import signal
import numpy as np
# import pandas as pd
# import copy
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# from sklearn.svm import SVC

# import pickle

def preproc_new(data, train_params, tau):

  a             = train_params['a']
  b             = train_params['b']
  fs            = train_params['fs']
  window        = train_params['window']
  numFreqOfSpec = train_params['numFreqOfSpec']
  nOver_win     = train_params['nOver_win']
  hamm_win      = train_params['hamm_win']
  min_seg       = train_params['min_seg']

  best_centers = []
  single_set   = []
  Y_data       = []

  for idx_gesture, gesture_data in enumerate(data):
    N_trial = len(gesture_data['data'])
    Y_data.append(idx_gesture+1)

    # Filtered signal
    filt_data = filt_emg(gesture_data['data'],b,a)

    # Detect muscle activity
    [seg_data,_,_,idx_s,idx_e] = muscle_activity(filt_data,
                                                 fs,
                                                 window,
                                                 numFreqOfSpec,
                                                 nOver_win,
                                                 hamm_win,
                                                 tau,
                                                 min_seg)

    # # Best center for each gesture
    # best_centers.append(best_center_class(seg_data))

    # Best center for each gesture using dtaidtw
    best_centers.append(best_center_dtai(seg_data))

    for d in seg_data:
      single_set.append(d)

  # Single set
  single_set = np.array(single_set, dtype=object)

  # Data label
  Y_data = np.repeat(np.array(Y_data), N_trial)

  return single_set, best_centers, Y_data

In [ ]:
# feature extractor

import numpy as np

def feature_extractor(data, best_centers):

  feat_data = []

  # Create the DTW between the input and the centers of each class
  for best_center in best_centers:
    dtw_data = dtw_extract(data, best_center)
    feat_data.append(dtw_data)

  feat_data = np.stack(feat_data, axis=1) # Convert the list of arrays to a single array

  # Normalizes the feature vector
  feat_data = feat_normalization(feat_data)

  return feat_data

In [ ]:
# training classifier

import numpy as np

def training_classifier(dataX_train, label, layers_dims, reg, mode):

  label_unique   = np.unique(label)  # How many unique labels are in the training dataset
  N_label_unique = len(label_unique) # How many classes are in total in the training dataset

  X_train = np.transpose(dataX_train)
  Y_train = np.eye(N_label_unique)[label-1]
  Y_train = np.transpose(Y_train)

  if mode == 'mlp_simple':
    parameters_tanh = L_layer_model(
                                    X_train,
                                    Y_train,
                                    layers_dims,
                                    learning_rate=0.01,
                                    num_iterations=50000,
                                    hidden_layers_activation_fn="tanh",
                                    lambda_reg=reg
                                    )

  else:
    raise ValueError("Parameter model must be 'mlp_simple'.")

  return parameters_tanh

In [ ]:
# new predictor

import numpy as np
# import pandas as pd
# import pickle

def predictor_new(data, best_centers, clf, params, tau, post='poll'):

  # Parameters definition
  a             = params['a']
  b             = params['b']
  fs            = params['fs']
  window        = params['window']
  numFreqOfSpec = params['numFreqOfSpec']
  nOver_win     = params['nOver_win']
  hamm_win      = params['hamm_win']
  min_seg       = params['min_seg']
  stride        = params['stride']
  win_len       = params['win_len']

  X_cache = []

  post_preds = []

  for gesture_data in data:
    X_cache_gesture = []

    post_preds_gestures = []

    for trial_data in gesture_data['data']:
      samples = len(trial_data) # Total samples in the trial

      start_point = 0
      win_count   = 0

      trial_preds = []

      X_cache_trial = []

      while True:
        start_point = win_count*stride
        end_point   = start_point + win_len

        # Exit if the time window pass the last trial's sample
        if end_point > samples:
          break

        # Selecting only a time window of size win_len from the trial
        windowed_data = trial_data[start_point:end_point]

        # Filtering the signal
        filt_data = filt_emg([windowed_data],b,a)

        # Detecting muscle activity
        [_,_,_,idx_s,idx_e] = muscle_activity(filt_data,
                                              fs,
                                              window,
                                              numFreqOfSpec,
                                              nOver_win,
                                              hamm_win,
                                              tau,
                                              min_seg)

        # Check if any muscle activity was detected
        # Otherwise, the prediction is no-gesture
        if idx_s[0] !=0 and idx_e[0] != len(windowed_data):
          windowed_data = windowed_data[idx_s[0]:idx_e[0]]
          filt_windowed_data = filt_emg([windowed_data],b,a)

          # Extracting and normalizing the features from the windowed data
          X = feature_extractor(filt_windowed_data, best_centers)
          X = np.transpose(X)

          # Saving a cache to decrease processing time when using ensemble
          X_cache_trial.append(X)

          # Fowarding the data into the net
          preds, caches = L_model_forward(X,
                                          clf,
                                          hidden_layers_activation_fn="tanh"
                                          )

          # We only consider predictions with confidence higher than 50%
          # Otherwise, the prediction is classified as no-gesture
          if max(preds) > 0.5:
            trial_preds.append(np.argmax(preds)+1)
          else:
            trial_preds.append(1)

        else:
          # If there is no muscle activity, the cache receives 0s
          X_cache_trial.append(np.zeros((6,1)))
          trial_preds.append(1)

        win_count += 1

      # Choose among the post processing methods
      if post == 'poll':
        post_pred = poll_postprocessing(trial_preds)
        post_preds_gestures.append(post_pred)

      elif post == 'base':
        post_pred = base_postprocessing(trial_preds)
        post_preds_gestures.append(post_pred)

      else:
        raise ValueError("The parameter post must be either 'poll' or 'base'.")

      X_cache_gesture.append(
          dict(
              trial = X_cache_trial
          )
      )

    post_preds.append(
        dict(
            preds = post_preds_gestures
        )
    )

    X_cache.append(
        dict(
            data = X_cache_gesture
        )
    )

  return post_preds, X_cache

In [ ]:
# new predictor for ensemble processing

import numpy as np

def predictor_ens_new(data, clf, post='poll'):

  post_preds = []

  for gesture_data in data:

    post_preds_gestures = []

    for trial_data in gesture_data['data']:

      trial_preds = []

      for window_data in trial_data['trial']:

        if window_data.all() != 0:
          X = window_data

          preds, caches = L_model_forward(X,
                                          clf,
                                          hidden_layers_activation_fn="tanh"
                                          )

          # We only consider predictions with confidence higher than 50%
          # Otherwise, the prediction is classified as no-gesture
          if max(preds) > 0.5:
            trial_preds.append(np.argmax(preds)+1)
          else:
            trial_preds.append(1)

        else:
          # If the window data in cache are all 0s, then predictions equal no-gesture
          trial_preds.append(1)

      # Choose among the post processing methods
      if post == 'poll':
        post_pred = poll_postprocessing(trial_preds)
        post_preds_gestures.append(post_pred)

      elif post == 'base':
        post_pred = base_postprocessing(trial_preds)
        post_preds_gestures.append(post_pred)

      else:
        raise ValueError("The parameter post must be either 'poll' or 'base'.")

    post_preds.append(
          dict(
              preds = post_preds_gestures
          )
      )

  return post_preds

In [ ]:
# poll post processing

def poll_postprocessing(preds):

  N_labels = len(preds)
  labels   = preds

  # Filter consecutive labels
  for label_i in range(1,N_labels):
    cond = preds[label_i] == preds[label_i - 1]
    labels[label_i]  = 1*cond + preds[label_i]*(1 - cond)

  if all(label == labels[0] for label in labels) and labels[0] == 1:
    pred = 1
  else:
    labels = [label for label in labels if label != 1] # Removing the no-gesture label
    pred   = max(set(labels), key=labels.count)

  return pred

In [ ]:
# baseline post processing

def base_postprocessing(preds):

  N_labels = len(preds)
  labels   = preds

  # Filter consecutive labels
  for label_i in range(1,N_labels):
    cond = preds[label_i] == preds[label_i-1]
    labels[label_i]  = 1*cond + preds[label_i]*(1 - cond)

  # Removing the no-gesture label
  unique_labels = list(set(labels))
  unique_labels = [label for label in unique_labels if label != 1]

  if len(unique_labels) == 0:
    pred = 1
  elif len(unique_labels) > 1:
    pred = unique_labels[0]
  else:
    pred = unique_labels

  return pred[0]

In [ ]:
# label extractor

def label_extractor(data, gestures):

  labels = []

  for gesture_data in data:
    label = gestures.index(gesture_data['gesture'])

    labels.append(
        dict(
            labels = [label+1]*len(gesture_data['data'])
        )
    )

  return labels

In [ ]:
# new accuracy

import numpy as np
from sklearn.metrics import confusion_matrix

def accuracy_new(preds, labels):

  labels = [label['labels'] for label in labels]  # Converting the label dict to a list
  labels = sum(labels, [])                        # Flattening the labels list

  preds = [pred['preds'] for pred in preds] # Converting the pred dict to a list
  preds = sum(preds, [])                    # Flattening the preds list

  # Creating the confusion matrix
  cm = confusion_matrix(labels,preds)

  # Calculating the accuracy
  acc = np.trace(cm)/np.sum(cm)

  return acc

In [ ]:
# Find the largest plateau

import numpy as np

def find_largest_plateau(arr):

  max_val = np.max(arr)
  max_indices = np.where(arr == max_val)[0]

  # Initialize variables to track the largest plateau
  start = max_indices[0]
  end = max_indices[0]
  max_length = 1

  current_start = max_indices[0]
  current_length = 1

  # Iterate over the indices of the max values to find continuous segments
  for i in range(1, len(max_indices)):

    if max_indices[i] == max_indices[i - 1] + 1:
      current_length += 1
      current_end = max_indices[i]

    else:
      current_start = max_indices[i]
      current_length = 1
      current_end = max_indices[i]

    # Update the max length and positions only if the current length is greater
    # or if it's equal to the max length but appears later
    if current_length >= max_length:
      max_length = current_length
      start = current_start
      end = current_end

  return arr[start:end+1], start, end

In [ ]:
# saving variables

import pickle

def save_var(var, name):

  with open(name+'.pkl', "wb") as f:
            pickle.dump(var, f)

In [ ]:
# main

# IMPORTING THE LIBRARIES
import numpy as np
from scipy import signal
from operator import itemgetter
import os

import warnings
warnings.filterwarnings("ignore")

In [ ]:
np.random.seed(1)
random.seed(100)

# Defining some constants
fs            = 200                                                              #Sample frequency
min_seg       = 100                                                              #Minimun segmentation length
numFreqOfSpec = 25                                                               #Number of frequencies for the STFT
hamm_win      = 25                                                               #Size of the Hamming window
nOver_win     = 10                                                               #Number of samples to overlap in the Hamming window
tau_u         = np.linspace(10,20,11)                                            #Constant to detect the muscle activity
filt_order    = 5                                                                #Filter of 5th order
filt_freq     = 0.1                                                              #Filter with cutoff frequency of 10Hz
stride        = 10
win_len       = 700
layers_dims   = [6, 8, 6]
reg           = 1e-2
N_ens         = 100
# tau_val       = np.array(([11., 13., 15., 15., 16., 18., 17., 10., 13., 19.]))
tau_val       = np.array(([11, 13., 15., 15., 16., 18., 13., 10., 13., 19.]))    # tau calculating by using fastdtw

# tau_val       = None
N_mhold       = 10
mode          = 'mlp_simple'
post          = 'poll'

validation = 'mhold'
val_split  = 0.2

b, a = signal.butter(filt_order, filt_freq, 'low')                               #Butterworth lowpass filter of "filt_order" order and cutoff frequency of "filt_freq" (normalized)

window = signal.windows.hamming(hamm_win)                                        #Hamming window

params = {'a':a,
          'b':b,
          'fs':fs,
          'min_seg':min_seg,
          'numFreqOfSpec':numFreqOfSpec,
          'hamm_win':hamm_win,
          'nOver_win':nOver_win,
          'window':window,
          'stride':stride,
          'win_len':win_len}

users = ['alejandroFlores',
        'alexandraApellido',
        'andresGuerra',
        'andresJaramillo',
        'cristhianMotoche',
        'dianitaCherrez',
        'homeroApellido',
        'jonathanZea',
        'juanYuquilema',
        'santiagoJaramillo']

# users = ['santiagoJaramillo']

gestures = ['relax',
            'fist',
            'wave_in',
            'wave_out',
            'fingers_spread',
            'double_tap']

In [ ]:
np.random.seed(1)
random.seed(100)

N_ens   = 100
# N_mhold = 5
# tau_u         = np.linspace(5,25,201)
# tau_u   = [17]

# mode = 'mlp_simple'
# post = 'poll'

# validation = 'mhold'
# val_split  = 0.2

X_val_ens  = []
X_test_ens = []

tau_best = []

acc_exp = []

version        = 'training'
raw_data_train = data_reader(users, gestures, version)
version        = 'testing'
raw_data_test  = data_reader(users, gestures, version)

# Splitting the data into training, validation and testing sets
data_splitted = set_split(raw_data_train, raw_data_test, validation, val_split)

for ens in range(N_ens):
  acc_users = []

  # Iterating over all users
  for idx_user, user in enumerate(users):

    # It is only necessary to compute the best tau in the firt ensemble iteration
    if ens > 0:
      tau_u = [tau_best[idx_user]]

    # Experimenting with specific tau_u values
    if tau_val is not None:
      tau_u = [tau_val[idx_user]]

    # Choose validation method
    if validation == 'mhold':

      net_fitted_tau  = []
      acc_tau         = []
      X_tau           = []

      # Validation process
      # Search for the best tau
      for tau in tau_u:

        net_fitted_val = []
        acc_val        = []

        # Computing the best centers for each class
        data_train = data_splitted[idx_user]['train']
        single_set, best_centers, dataY_train = preproc_new(data_train, params, tau)

        # Extracting and normalizing the features from the data
        feat_dataX = feature_extractor(single_set, best_centers)

        # Choose the best net among those trained by the multi-holdout method
        for mhold_n in range(N_mhold):

          # Fit the network
          net_fitted_mhold = training_classifier(feat_dataX, dataY_train, layers_dims, reg, mode)
          # Store the network fitted in this mhold iteration
          net_fitted_val.append(net_fitted_mhold)

          # Predicting in the validation set
          # In the first iteration of mhold, compute the X
          # In subsequent iterations of mhold, use X_cache to speed the process
          if mhold_n == 0 and ens == 0:
            data_val = data_splitted[idx_user]['val']
            predictions_new, X_val_cache = predictor_new(data_val,
                                                        best_centers,
                                                        net_fitted_mhold,
                                                        params,
                                                        tau,
                                                        post=post)

            X_tau.append(X_val_cache)

          else:
            if ens > 0:
              X_val_cache = X_val_ens[idx_user]

            predictions_new = predictor_ens_new(X_val_cache,
                                                net_fitted_mhold,
                                                post=post)

          # Extracting the labels from the validation data
          labels = label_extractor(data_val, gestures)

          # Computing the validation accuracy
          acc_mhold = accuracy_new(predictions_new, labels)
          # Store the accuracy computed in this mhold iteration
          acc_val.append(acc_mhold)
          # print(f'{user} reached accuracy of {acc_mhold} for tau_u={tau}, iteration {mhold_n}')

        # Computing the maximum validation score considering the iterations of mhold
        acc_val_max = max(acc_val)
        i_acc_val_max = acc_val.index(acc_val_max)

        # Store the net associated with the maximum validation score
        net_fitted_tau.append(net_fitted_val[i_acc_val_max])
        # Store the maximum validation score
        acc_tau.append(acc_val_max)
        # print(f'{user} reached best accuracy for tau_u={tau} is {acc_val_max} from iteration {i_acc_val_max}')

      # Computing the maximum validation score considering the maximum score in each tau
      acc_tau_max = max(acc_tau)
      _, start_acc_tau_max, end_acc_tau_max = find_largest_plateau(acc_tau)
      i_acc_tau_max = int(np.ceil((end_acc_tau_max-start_acc_tau_max)/2)+start_acc_tau_max)
      # i_acc_tau_max = acc_tau.index(acc_tau_max)

      # Store tau and X_cache only in the first ensemble iteration
      if ens == 0:
        # Store the best tau associated with the maximum validation score
        tau_best.append(tau_u[i_acc_tau_max])
        print(f'The best tau for {user} is {tau_best[idx_user]}')
        # Store the X_cache associated with the best tau
        X_val_ens.append(X_tau[i_acc_tau_max])

      # Store the net associated with the best tau
      net_fitted_best = net_fitted_tau[i_acc_tau_max]


    # Predicting in the testing set
    # In the first iteration of the ensemble, computes the X
    # In subsequent iterations, uses X_cache to speed the process
    if ens == 0:
      data_test = data_splitted[idx_user]['test']
      predictions_new, X_test_cache = predictor_new(data_test,
                                                    best_centers,
                                                    net_fitted_best,
                                                    params,
                                                    tau_best[idx_user],
                                                    post=post)

      X_test_ens.append(X_test_cache)

    else:
      predictions_new = predictor_ens_new(X_test_ens[idx_user],
                                          net_fitted_best,
                                          post=post)

    # Extracting the labels from the test data
    labels = label_extractor(data_test, gestures)

    # Computing the test accuracy
    acc_user = accuracy_new(predictions_new, labels)

    acc_users.append(acc_user)

  acc_exp.append(acc_users)
  print(f'Accuracy in ensemble {ens} was {np.mean(acc_users)}')

The best tau for alejandroFlores is 11.0
The best tau for alexandraApellido is 15.0
The best tau for andresGuerra is 15.0
The best tau for andresJaramillo is 16.0
The best tau for cristhianMotoche is 15.0
The best tau for dianitaCherrez is 13.0
The best tau for homeroApellido is 17.0
The best tau for jonathanZea is 15.0
The best tau for juanYuquilema is 18.0
The best tau for santiagoJaramillo is 17.0
Accuracy in ensemble 0 was 0.9824999999999999
Accuracy in ensemble 1 was 0.9666666666666666
Accuracy in ensemble 2 was 0.9691666666666666
Accuracy in ensemble 3 was 0.9766666666666668
Accuracy in ensemble 4 was 0.9716666666666667
Accuracy in ensemble 5 was 0.9641666666666666
Accuracy in ensemble 6 was 0.9724999999999999
Accuracy in ensemble 7 was 0.9783333333333333
Accuracy in ensemble 8 was 0.9724999999999999
Accuracy in ensemble 9 was 0.9708333333333334
Accuracy in ensemble 10 was 0.9791666666666667
Accuracy in ensemble 11 was 0.9691666666666666
Accuracy in ensemble 12 was 0.975
Accuracy

In [ ]:
# tau_best = fast
for i in range(10):
  print(np.mean([acc[i] for acc in acc_exp]))


0.9916666666666667
0.9388333333333333
0.9405833333333331
0.9954999999999999
0.9804166666666666
0.9306666666666668
0.9329999999999998
1.0
0.9988333333333334
1.0
